In [1]:
import gym
import os
import numpy as np
import tensorflow as tf
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
env = gym.make('BipedalWalkerHardcore-v2')
obs = env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
obs

array([ 2.74583930e-03,  1.04752497e-05, -1.36591885e-03, -1.60000670e-02,
        9.24866349e-02,  3.17089702e-03,  8.59792963e-01, -1.11846168e-03,
        1.00000000e+00,  3.28033380e-02,  3.17077059e-03,  8.53535444e-01,
       -2.15749846e-03,  1.00000000e+00,  4.40813392e-01,  4.45819497e-01,
        4.61422116e-01,  4.89549488e-01,  5.34102023e-01,  6.02460206e-01,
        7.09147871e-01,  8.85930538e-01,  1.00000000e+00,  1.00000000e+00])

In [4]:
possible_torques = np.array([-1.0,-0.5, 0.0, 0.5, 1.0])
possible_actions = np.array(list(product(possible_torques, possible_torques, possible_torques, possible_torques)))
possible_actions.shape

(625, 4)

In [5]:
tf.reset_default_graph()

# 1. Specify the network architecture
n_inputs = env.observation_space.shape[0]  # == 24
n_hidden = 16
n_outputs = len(possible_actions) # == 81
initializer = tf.variance_scaling_initializer()

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.selu, kernel_initializer=initializer)
hidden = tf.layers.dense(hidden, n_hidden, activation=tf.nn.selu, kernel_initializer=initializer)
hidden = tf.layers.dense(hidden, n_hidden, activation=tf.nn.selu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs, kernel_initializer=initializer)
outputs = tf.nn.softmax(logits)

# 3. Select a random action based on the estimated probabilities
action_index = tf.squeeze(tf.multinomial(logits, num_samples=1), axis=-1)

# 4. Training
learning_rate = 0.001

y = tf.one_hot(action_index, depth=len(possible_actions))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [6]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [ ]:
n_games_per_update = 1
n_max_steps = 1000
n_iterations = 50000
save_iterations = 10
discount_rate = 0.99
highest_me = -220
total_rewards =[]
mean_scores = [] 
me = -300
with tf.Session() as sess:
    init.run()
#     saver.restore(sess, "./models/my_bipedal_walker_pg.ckpt")
    for iteration in range(n_iterations):
        if len(mean_scores)>=100:
            me =  (sum(mean_scores[-100:])/100)
            print("\rIteration: {}/{}, Mean score: {}, Best ME: {}".format(iteration + 1, n_iterations, me, highest_me), end="")
        else:
            print("\rIteration: {}/{}".format(iteration + 1, n_iterations), end="")
        all_rewards = []
        all_gradients = []
        mean_scores = mean_scores[-101:]
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            score = 0
            for step in range(n_max_steps):
                action_index_val, gradients_val = sess.run([action_index, gradients],
                                                           feed_dict={X: obs.reshape(1, 24)})
                
                action = possible_actions[action_index_val]
                obs, reward, done, info = env.step(action[0])
                score+=reward
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
            mean_scores.append(score)
            
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
        feed_dict = {}
        for var_index, gradient_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                          for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./models/my_bipedal_walker_pg.ckpt")
        if highest_me < me:
            highest_me = me
            saver.save(sess, "./models/best_walker.ckpt")

Iteration: 28341/50000, Mean score: -99.91848103405408, Best ME: -98.96198488662299